In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import re
import requests
import holoviews as hv
import matplotlib.pyplot as plt
import pint
import nd2reader
import tifffile
import scipy.stats

In [ ]:
%load_ext autoreload
%autoreload 2
import simulation

In [ ]:
u = simulation.ureg

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
excitation_bins = np.linspace(300, 850, 1000)
emission_bins = excitation_bins

# Sample

In [ ]:
# nd2 = nd2reader.ND2Reader("ClpP_mEGFP_100x.nd2")
# nd2._parser._raw_metadata.z_levels = [0]
# sample_img = nd2.get_frame_2D(v=0, c=1)  # [:,500:1000]
# sample_img = image_to_xarray(sample_img, nd2.metadata["pixel_microns"])

In [ ]:
sample_img = tifffile.imread("t000001xy14c2.tif")
scale = 4.25 / 20
sample_img = simulation.image_to_xarray(sample_img, scale)

In [ ]:
sample_img.plot.imshow()

# Excitation line

In [ ]:
line_img = simulation.draw_excitation_line(1.3 * u.mm, 0.2 * u.um, 0.2 * u.um, 0.3)

In [ ]:
sample_offset = simulation.offset_xarray(sample_img, line_img, dict(x=500, y=470))

In [ ]:
(sample_offset * line_img).plot(aspect=20, size=5)